In [3]:
import os
import pandas as pd
from datetime import datetime as dt
from pathlib import Path

In [4]:
# import pytz
from datetime import timezone, timedelta
# dt_utc_aware = datetime.datetime.fromtimestamp(0, datetime.timezone.utc)

In [5]:
# new_timezone = pytz.timezone("US/Eastern")

In [14]:
from datetime import timezone, timedelta
def get_weather_pandas(path_to_csv: str) -> pd.DataFrame:
    weather_data = pd.read_csv(path_to_csv)
    
    # Unix timestamp to UTC-5
    weather_data['time'] = weather_data['time'].apply(lambda x: (dt.fromtimestamp(x, timezone.utc) - timedelta(hours=5)).strftime('%Y-%m-%d %H:%M:%S'))
    
    weather_data.rename(columns={'time' : 'timestamp'}, inplace=True)
    weather_data = weather_data.set_index('timestamp')
    weather_data.index = pd.to_datetime(weather_data.index, format='%Y-%m-%d %H:%M:%S')
    
    assert weather_data[weather_data.index.duplicated()].empty # making sure there's no duplicates

    weather_data = weather_data[['temperature', 'humidity']]
    weather_data['temperature'] = weather_data['temperature'].apply(lambda x: (x - 32) / 1.8)
    weather_data['humidity'] = weather_data['humidity'].apply(lambda x: x * 100)
    
    return weather_data

In [26]:
dataset_path = Path(os.environ.get('BUILDINGS_BENCH', ''))

home = [('HomeB',[2014,2015,2016]), ('HomeC', [2014, 2015, 2016]), ('HomeD', [2015, 2016]), ('HomeF', [2014, 2015, 2016]), ('HomeG', [2015, 2016])]
home

In [ ]:
for h in home:
    weather_df = pd.DataFrame()
    for y in h[1]:
        df = get_weather_pandas(f'{h[0]}/{h[0][0].lower() + h[0][1:]}{y}.csv')
        weather_df = pd.concat([weather_df, df])

    weather_df.to_csv(dataset_path / f'remove_outliers/SMART/weather_{h[0]}.csv')